In [1]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import col

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

#!hdfs dfs -ls
!hdfs dfs -copyFromLocal Asistencia.csv /tmp/
!hdfs dfs -copyFromLocal Evaluacion.csv /tmp/

In [3]:
!hdfs dfs -ls /tmp/*csv

-rw-r--r--   1 root supergroup      93121 2021-05-27 15:43 /tmp/Asistencia.csv
-rw-r--r--   1 root supergroup     111944 2021-05-27 15:43 /tmp/Evaluacion.csv


In [4]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...
asis = SesionSpark.read.load('/tmp/Asistencia.csv',format="csv",sep=',',inferSchema='true',header='true')
eva = SesionSpark.read.load('/tmp/Evaluacion.csv',format="csv",sep=',',inferSchema='true',header='true')

In [5]:
#Validamos datos cambiando el nombre de las columnas
asis1 = asis.withColumnRenamed("Marca temporal", "tiempo")
asis2 = asis1.withColumnRenamed("Nombre de usuario", "usuario")
asis3 = asis2.withColumnRenamed("Equipo al que perteneces:", "equipoPertenece")
asis4 = asis3.withColumnRenamed("Equipo que va a exponer:", "equipoExponer")
asis4.printSchema()
asis = asis4

root
 |-- tiempo: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- equipoPertenece: string (nullable = true)
 |-- equipoExponer: string (nullable = true)



In [6]:
eva2 = eva.withColumnRenamed("Marca temporal","tiempo") \
    .withColumnRenamed("Nombre de usuario","usuario") \
    .withColumnRenamed("Equipo que vas a evaluar:","equipoEvaluar") \
    .withColumnRenamed("Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?","ventajas") \
    .withColumnRenamed("Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?","problema") \
    .withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","introduccion") \
    .withColumnRenamed("Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?","solucion") \
    .withColumnRenamed("Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.","traccion") \
    .withColumnRenamed("Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ","modeloNegocio") \
    .withColumnRenamed("Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?","inversion") \
    .withColumnRenamed("Exposición: ¿Qué tan coordinados estaban los expositores?","exposicion1") \
    .withColumnRenamed("Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?","exposicion2") \
    .withColumnRenamed("Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.","exposicion3") \
    .withColumnRenamed("Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)","supon") \
    .withColumnRenamed("Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.","observacion") \
    .withColumnRenamed("Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.","contacto") \
    .withColumnRenamed("Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?","competencia") \
    .withColumnRenamed("Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.","mercado") \
    .withColumnRenamed("Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.","producto") \
    .withColumnRenamed("Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?", "equipo")
eva = eva2
eva2.printSchema()

root
 |-- tiempo: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- equipoEvaluar: string (nullable = true)
 |-- introduccion: integer (nullable = true)
 |-- equipo: integer (nullable = true)
 |-- problema: integer (nullable = true)
 |-- ventajas: integer (nullable = true)
 |-- solucion: integer (nullable = true)
 |-- producto: integer (nullable = true)
 |-- traccion: integer (nullable = true)
 |-- mercado: integer (nullable = true)
 |-- competencia: integer (nullable = true)
 |-- modeloNegocio: integer (nullable = true)
 |-- inversion: integer (nullable = true)
 |-- contacto: integer (nullable = true)
 |-- exposicion1: integer (nullable = true)
 |-- exposicion2: integer (nullable = true)
 |-- exposicion3: integer (nullable = true)
 |-- supon: string (nullable = true)
 |-- observacion: string (nullable = true)



In [7]:
#Datos adicionales
totalAsis = asis.count()
totalEva = eva.count()
print('Total de asistencias: ', totalAsis, ' y total de evaluaciones: ', totalEva, '.')

Total de asistencias:  662  y total de evaluaciones:  617 .


In [8]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

num = asis.sort(['usuario']).select(['usuario']).drop_duplicates().count()
#Respuesta
asis.sort(['usuario']).select(['usuario']).drop_duplicates().show(num)

+--------------------+
|             usuario|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
|cquinchiar@unal.e...|
| dadazam@unal.edu.co|
|daestradam@unal.e...|
|davgarciava@unal....|
|dballesteroso@una...|
|  dbrito@unal.edu.co|
|dcadavid@unal.edu.co|
|dcardonaal@unal.e...|
|dchavarriar@unal....|
|dgarciabl@unal.ed...|
|dgerenal@unal.edu.co|
|dgiraldolo@unal.e...|
|dguardia@unal.edu.co|
|dosoriom@unal.edu.co|
| eapenad@unal.edu.co|
|elondonoc@unal.ed...|
|emflorezb@unal.ed...|
|emlopezr@unal.edu.co|
|eporrasm@unal.edu.co|
|esgarciac@unal.ed...|
|favasquezj@unal.e...|
|fguerrerot@unal.e...|
|fmiranda@unal.edu.co|
|framirezf@unal.ed...|
|hriveraa@unal.edu.co|
|jcardonaso@unal.e...|
|jcorreapa@unal.ed...|
|jdroldano@unal.ed...|
|jfernandezmo@unal...|
|jgutierrezlo@unal...|
|jhcordoba@

In [9]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

#Creamos una vista para trabajar con SQL
asis.createOrReplaceTempView('vistaAsis')

from pyspark.sql import functions as F

#Respuesta
SesionSpark.sql("""
select
usuario,
count(usuario) as numAsistencia,
max(tiempo) as ultima_asistencia,
min(tiempo) as primera_asistencia
from
vistaAsis
group by
usuario

""").show()

+--------------------+-------------+--------------------+--------------------+
|             usuario|numAsistencia|   ultima_asistencia|  primera_asistencia|
+--------------------+-------------+--------------------+--------------------+
|samolinap@unal.ed...|           10|2021/04/13 9:37:2...|2021/04/08 8:11:4...|
| jruedat@unal.edu.co|            8|2021/04/13 9:41:2...|2021/04/08 8:12:5...|
|nvalenciat@unal.e...|            8|2021/04/13 9:39:3...|2021/04/08 8:10:1...|
|serendona@unal.ed...|            8|2021/04/13 9:40:2...|2021/04/08 8:10:4...|
|jgutierrezlo@unal...|            8|2021/04/13 9:39:0...|2021/04/08 8:12:0...|
|dgerenal@unal.edu.co|            8|2021/04/13 9:41:2...|2021/04/08 8:10:0...|
|cjfunezg@unal.edu.co|            8|2021/04/13 9:41:4...|2021/04/08 8:12:1...|
|emflorezb@unal.ed...|            8|2021/04/13 9:45:4...|2021/04/08 8:14:0...|
|dchavarriar@unal....|            8|2021/04/13 9:43:5...|2021/04/08 8:13:0...|
|fguerrerot@unal.e...|            8|2021/04/13 9:40:

In [18]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select usuario, total from (select usuario, count(usuario) as total from vistaAsis group by usuario) where total <= 2
""").show()

+--------------------+-----+
|             usuario|total|
+--------------------+-----+
|jdroldano@unal.ed...|    1|
+--------------------+-----+



In [12]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

#Creamos otra vista para trabajar con SQL
eva.createOrReplaceTempView('vistaEva')

In [13]:
#Respuesta
SesionSpark.sql("""
 select usuario, total from (select usuario, count(usuario) as total from vistaAsis group by usuario) where total == 0
""").show()

+-------+-----+
|usuario|total|
+-------+-----+
+-------+-----+



In [14]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select SUBSTRING(equipoPertenece, 14) as equipos, collect_set(usuario) as integrantes from vistaAsis group by equipoPertenece
""").show()

+--------------------+--------------------+
|             equipos|         integrantes|
+--------------------+--------------------+
| Gestión de Menús...|[lmazo@unal.edu.c...|
| Lo tengo y te lo...|[dgarciabl@unal.e...|
| Administración d...|[kmolano@unal.edu...|
| Servicios a domi...|[jruedat@unal.edu...|
|       Te lo cambio.|[fguerrerot@unal....|
| Gestión de Resid...|[mcarvajalsa@unal...|
| Gestión de progr...|[acastrillonv@una...|
|     Mi Sanandresito|[juochoag@unal.ed...|
+--------------------+--------------------+



In [15]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select SUBSTRING(equipoPertenece, 14) as Equipo, count(usuario) as CantidadAsistentes from vistaAsis where equipoPertenece <> equipoExponer group by equipoPertenece 
""").show()

+--------------------+------------------+
|              Equipo|CantidadAsistentes|
+--------------------+------------------+
| Gestión de Menús...|                84|
| Lo tengo y te lo...|                87|
| Administración d...|                95|
| Servicios a domi...|                65|
|       Te lo cambio.|                89|
| Gestión de Resid...|                86|
| Gestión de progr...|                73|
|     Mi Sanandresito|                83|
+--------------------+------------------+



In [17]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select * from (select A.usuario, SUBSTRING(A.equipoPertenece, 14) AS EquipoPertenece, E.equipoEvaluar from vistaAsis A inner join vistaEva E on A.usuario = E.usuario) where EquipoPertenece = equipoEvaluar
""").show()

+-------+---------------+-------------+
|usuario|EquipoPertenece|equipoEvaluar|
+-------+---------------+-------------+
+-------+---------------+-------------+



In [35]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

#Respuesta
nota = SesionSpark.sql("""

--Creamos columna con la nota final
SELECT *,
ROUND(
(
(
promIntroduccion +
promEquipo +
promProblema +
promVentajas +
promSolucion +
promProducto + 
promTraccion +
promMercado +
promCompetencia +
PromModeloNegiocio +
promInversion +
promContacto +
promExposicion1 +
promExposicion2 +
promExposicion3
) / 15.0), 2
) 
AS Nota_Final

FROM

(
-- Promedio de cada columna
SELECT 
equipoEvaluar,
ROUND(avg(introduccion),2) as promIntroduccion,
ROUND(avg(equipo),2) as promEquipo,
ROUND(avg(problema),2) as promProblema,
ROUND(avg(ventajas),2) as promVentajas,
ROUND(avg(solucion),2) as promSolucion,
ROUND(avg(producto),2) as promProducto,
ROUND(avg(traccion),2) as promTraccion,
ROUND(avg(mercado),2) as promMercado,
ROUND(avg(competencia),2) as promCompetencia,
ROUND(avg(modeloNegocio),2) as PromModeloNegiocio,
ROUND(avg(inversion),2) as promInversion,
ROUND(avg(contacto),2) as promContacto,
ROUND(avg(exposicion1),2) as promExposicion1,
ROUND(avg(exposicion2),2) as promExposicion2,
ROUND(avg(exposicion3),2) as promExposicion3 

FROM

(
-- Evaluaciones de cada miembro a un equipo diferente al que pertenece
SELECT *
FROM

(
-- Tomar datos del documento evaluación, de acuerdo al equipo que pertenecen
SELECT *,

(
-- equipo al que pertenecen
SELECT SUBSTRING(equipoPertenece,15) AS equipoPertenece 
FROM vistaAsis as A 
WHERE A.usuario = usuario LIMIT 1
)

AS equipoPertenece 
from vistaEva
) 

WHERE equipoPertenece != equipoEvaluar
)

GROUP BY equipoEvaluar
)


""")

nota.show()

+--------------------+----------------+----------+------------+------------+------------+------------+------------+-----------+---------------+------------------+-------------+------------+---------------+---------------+---------------+----------+
|       equipoEvaluar|promIntroduccion|promEquipo|promProblema|promVentajas|promSolucion|promProducto|promTraccion|promMercado|promCompetencia|PromModeloNegiocio|promInversion|promContacto|promExposicion1|promExposicion2|promExposicion3|Nota_Final|
+--------------------+----------------+----------+------------+------------+------------+------------+------------+-----------+---------------+------------------+-------------+------------+---------------+---------------+---------------+----------+
| permitiendo hace...|            null|      null|        null|        null|        null|        null|        null|       null|           null|              null|         null|        null|           null|           null|           null|      null|
| po

In [ ]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

##Se consulta por ítem los equipos que presentan el promedio igual al mayor promedio de cada categoría y total
##Se unen las consultas en una sola tabla

ContextoSql.sql("""

SELECT R.Equipo_evaluar, R.Avgintroduccion AS item, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgintroduccion) AS Nota FROM resultado) X on R.Avgintroduccion = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgequipo, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgequipo) AS Nota FROM resultado) X on R.Avgequipo = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgproblema, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgproblema) AS Nota FROM resultado) X on R.Avgproblema = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgventajas, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgventajas) AS Nota FROM resultado) X on R.Avgventajas = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgsolucion, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgsolucion) AS Nota FROM resultado) X on R.Avgsolucion = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgproducton, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgproducton) AS Nota FROM resultado) X on R.Avgproducton = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgtraccion, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgtraccion) AS Nota FROM resultado) X on R.Avgtraccion = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgmercado, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgmercado) AS Nota FROM resultado) X on R.Avgmercado = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgcompetencia , X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgcompetencia ) AS Nota FROM resultado) X on R.Avgcompetencia  = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgmodelo_negocio, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgmodelo_negocio) AS Nota FROM resultado) X on R.Avgmodelo_negocio = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avginversion, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avginversion) AS Nota FROM resultado) X on R.Avginversion = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgcontacto, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgcontacto) AS Nota FROM resultado) X on R.Avgcontacto = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgexp_coordinacion, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgexp_coordinacion) AS Nota FROM resultado) X on R.Avgexp_coordinacion = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgexp_claridad, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgexp_claridad) AS Nota FROM resultado) X on R.Avgexp_claridad = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.Avgexp_diapositivas, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(Avgexp_diapositivas) AS Nota FROM resultado) X on R.Avgexp_diapositivas = X.Nota

---------------

UNION ALL

---------------

SELECT R.Equipo_evaluar, R.NotaFinal, X.Nota As Mejor_Promedio

FROM resultado R INNER JOIN (SELECT MAX(NotaFinal) AS Nota FROM resultado) X on R.NotaFinal = X.Nota


 """).show()

In [ ]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...



In [ ]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

